In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re

In [2]:
data=pd.read_csv("../input/samplcsv/sample-2.csv")

In [3]:
newdata=pd.read_csv("../input/oathss/oaths.csv")

In [4]:
data["transcription_text"][10]

'[[image - eagle]]\r\nAMNESTY OATH.\r\n\r\nMississippi City Miss Sept 29 1865.\r\n\r\nI, J.W. Baxter, do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. SO HELP ME GOD.\r\nJ.W. Baxter\r\n\r\nSworn to and subscribed at Mississippi City\r\nthis 29th day of September 1865, before me\r\nJ. H. Mathews Capt 66th\r\nU.S Col Infty & Provost Marshal.  Sub port Miss\r\n\r\nDESCRIPTION:\r\nAge 24 years\r\nHeight 5ft 8 inches\r\nEyes Blue\r\nColor of Hair light\r\nCharacteristics [[blank]]\r\n\r\nHEADQUARTERS, Mississippi City Miss\r\nSept 29th 1865.\r\n\r\nI hereby certify, that the Amnesty Oath prescribed by the Proclamation of the President of the United States, was duly adm

In [5]:
txte= re.sub("([\(\[]).*?([\)\]])","",data["transcription_text"][10])
txte

']\r\nAMNESTY OATH.\r\n\r\nMississippi City Miss Sept 29 1865.\r\n\r\nI, J.W. Baxter, do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. SO HELP ME GOD.\r\nJ.W. Baxter\r\n\r\nSworn to and subscribed at Mississippi City\r\nthis 29th day of September 1865, before me\r\nJ. H. Mathews Capt 66th\r\nU.S Col Infty & Provost Marshal.  Sub port Miss\r\n\r\nDESCRIPTION:\r\nAge 24 years\r\nHeight 5ft 8 inches\r\nEyes Blue\r\nColor of Hair light\r\nCharacteristics ]\r\n\r\nHEADQUARTERS, Mississippi City Miss\r\nSept 29th 1865.\r\n\r\nI hereby certify, that the Amnesty Oath prescribed by the Proclamation of the President of the United States, was duly administered to J.W. Baxter

# Preprocessing

In [6]:
text=data["transcription_text"]

In [7]:
all_text=newdata["transcription_text"]

In [8]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()

In [9]:
stop_words.remove("of")
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punctuation:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

In [11]:
for i in tqdm(range(data.shape[0])):
    text[i] = process(text[i])

100%|██████████| 20/20 [00:01<00:00, 11.00it/s]


In [12]:
for i in tqdm(range(newdata.shape[0])):
    all_text[i] = process(all_text[i])

100%|██████████| 2136/2136 [00:04<00:00, 470.05it/s]


In [13]:
for i in tqdm(range(data.shape[0])):
    text[i] = " ".join(text[i])

100%|██████████| 20/20 [00:00<00:00, 1489.59it/s]


In [14]:
for i in tqdm(range(newdata.shape[0])):
    all_text[i] = " ".join(all_text[i])

100%|██████████| 2136/2136 [00:00<00:00, 2928.50it/s]


In [15]:
all_text

0       I I C Meekins of Currituck County State of Nor...
1       Department of Virginia North Carolina 1864 tak...
2       I W H Pearce of Craven County State of NorthCa...
3       State of North Carolina County I solemnly swea...
4       State of North Carolina Wake County I SE Allen...
                              ...                        
2131    Office Provost Marshal Raleigh NC May 26 65 Th...
2132    July 30th 1867 Birnie Wm Agent Bu R F A L Oath...
2133    State of North Carolina Robeson County I Willi...
2134    NARA 449 102 We whose name written well person...
2135    NARA 450 404 Office Provost Marshall City of W...
Name: transcription_text, Length: 2136, dtype: object

In [16]:
text

0     8 E encl I Anne H Elliott widow solemnly swear...
1     I E C Whaley solemnly swear I carefully read A...
2     43M I Mrs E Montgomery solemnly Swear I carefu...
3     No 2324 I solemnly swear affirm presence of Al...
4     August 24 1865 APPLICATION AND CERTIFICATE OF ...
5     I Wm Whaley solemnly swear I carefully read Am...
6     I J F Steinmeyer solemnly swear I carefully re...
7     I Joseph Edings solemnly swear I carefully rea...
8     Vicksburg Miss July 22 1867 Thomas Andrew Oath...
9     455 I Jacob Schroder solemnly swear I carefull...
10    AMNESTY OATH Mississippi City Miss Sept 29 186...
11    OATH I P P Bergevin solemnly swear I never vol...
12                                         Frank Conner
13    I Caroline Belitzer solemnly Swear I carefully...
14    Triplicate AMNESTY OATH Pass Christian Miss Au...
15    HEADQUARTERS UNITED STATES FORCES No 1282 PROV...
16                                      Robert A Wilson
17    APPLICATION AND CERTIFICATE OF James D Ken

# Flair

In [17]:
!pip3 install Cython
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 46.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-p

# Hugging face  : Flair

**ner-english** 

In [18]:
!git lfs install
!git clone https://huggingface.co/flair/ner-english-large

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log
Cloning into 'ner-english-large'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), 11.03 KiB | 1.10 MiB/s, done.


In [19]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-08 11:46:20,917 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-08 11:46:48,351 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [20]:
from flair.models import SequenceTagger

tagger1 = SequenceTagger.load("flair/ner-english")

Downloading:   0%|          | 0.00/432M [00:00<?, ?B/s]

2022-06-08 11:47:09,111 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-06-08 11:47:11,292 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [21]:
from flair.models import SequenceTagger

tagger2 = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-08 11:48:50,444 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-06-08 11:49:07,584 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


tagger3 nerenglishlarge

In [22]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
from flair.data import Sentence

dflarge=pd.DataFrame({"text":"", "PERSON-nerenglarge":[], "LOC-nerenglarge":"","ORG-nerenglarge":""})
for i in range(20):
        sentencess =Sentence(all_text[i])
        dflarge.loc[i,"text"]=all_text[i]
        tagger2.predict(sentencess)
        dc={}
        P=[]
        L=[]
        G=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        lst.append(entity)
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") | (dc[j]=="MISC") :
                      O.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
            

dflarge["PERSON-nerenglarge"]=Per
dflarge["LOC-nerenglarge"]=LOC
dflarge["ORG-nerenglarge"]=ORG                        

In [23]:
dflarge

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,I I C Meekins of Currituck County State of Nor...,"[I I C Meekins, God, Isaac C Meekins, E H Walk...","[Currituck County State of North Carolina, Uni...",[]
1,Department of Virginia North Carolina 1864 tak...,[],"[North Carolina, United States, States]",[Department of Virginia]
2,I W H Pearce of Craven County State of NorthCa...,"[W H Pearce, Almighty, God, Jm E Ampetts, A B ...","[Craven County State of NorthCarolina, United ...",[]
3,State of North Carolina County I solemnly swea...,[God],"[State of North Carolina County, United States]",[]
4,State of North Carolina Wake County I SE Allen...,"[SE Allen, God SE Allen Sworn, McAlpine]","[State of North Carolina, Wake County, United ...",[Res Corp]
5,I Sarah E Dough of Currituck county State of N...,"[Sarah E Dough, God, Isaac C Merkins]","[Currituck, State of North Carolina, United St...",[]
6,341 Department of Virginia North Carolina Roan...,"[Sarah E Dough, JM Wilson]","[North Carolina, Roanoke Island, NC, United St...","[Department of Virginia, General Orders No 49]"
7,I W T Dough of Currituck County State of North...,"[I, W T Dough, God, E H Walker, John B Etheridge]","[Currituck County State of NorthCarolina, Unit...",[Constitution]
8,I Jesse E Dough of Currituck County State of N...,"[Jesse E Dough, Almighty, God, Isaac C Meekins...","[Currituck County State of North Carolina, Uni...",[]
9,I T A Dough of Currituck County State of North...,"[I T A Dough, God, John B Etheridge, T A Dough]","[Currituck County State of NorthCarolina, Unit...",[]


tagger1 ner english

In [24]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
from flair.data import Sentence

dfner=pd.DataFrame({"text":"", "PERSON-nerenglish":"","LOC-nerenglish":"","ORG-nerenglish":[]})
for i in range(20):
        sentencess =Sentence(all_text[i])
        dfner.loc[i,"text"]=all_text[i]
        tagger1.predict(sentencess)
        dc={}
        P=[]
        L=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") | (dc[j]=="MISC") :
                      O.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
            

dfner["PERSON-nerenglish"]=Per
dfner["LOC-nerenglish"]=LOC
dfner["ORG-nerenglish"]=ORG                        

In [25]:
dfner.head()

,text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish
0,I I C Meekins of Currituck County State of Nor...,[John B Etheridge],[],[]
1,Department of Virginia North Carolina 1864 tak...,[],[United States],[]
2,I W H Pearce of Craven County State of NorthCa...,"[H Pearce, B Watson]",[],[]
3,State of North Carolina County I solemnly swea...,[],[United States],[]
4,State of North Carolina Wake County I SE Allen...,[],[United States],[]


tagger 2 ontonotes

In [26]:
lst=[]
label=[]
Per=[]
Date=[]
GPE=[]
ORG=[]
from flair.data import Sentence

aldf=pd.DataFrame({"text":"", "PERSON-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})
for i in range(2136):
        sentencess =Sentence(all_text[i])
        aldf.loc[i,"text"]=all_text[i]
        tagger.predict(sentencess)
        dc={}
        P=[]
        D=[]
        G=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.98) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        lst.append(entity)
        
        
        for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
        Per.append(P)
        Date.append(D)
        ORG.append(O)
        GPE.append(G)
            
aldf["Date-ontonotes"]=Date
aldf["PERSON-ontonotes"]=Per
aldf["GPE-ontonotes"]=GPE
aldf["ORG-ontonotes"]=ORG    

In [27]:
aldf.shape

(2136, 5)

In [28]:
aldf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes
0,I I C Meekins of Currituck County State of Nor...,"[I I C Meekins, Isaac C Meekins, E H Walker JP...",[North Carolina],[],[16 day of Sept AD 1865]
1,Department of Virginia North Carolina 1864 tak...,[],"[North Carolina, United States]",[Department of Virginia],[1864]
2,I W H Pearce of Craven County State of NorthCa...,"[W H Pearce, Watson]",[],[],[3rd day of October AD 1865]
3,State of North Carolina County I solemnly swea...,[],"[North Carolina County, United States]",[],[]
4,State of North Carolina Wake County I SE Allen...,[Allen],"[North Carolina, Wake County, United States]",[],[31st day of May 1867]


In [29]:
aldf.shape

(2136, 5)

# Post processing

**For dates**

In [30]:
G=[]
for i in aldf["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-ZZa-z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)

    
aldf["postDate-ontonotes"]=GF
            
            
        

In [31]:
aldf.shape

(2136, 6)

In [32]:
G=[]
for i in aldf["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Z][a-z]\S*)\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        final_date4=re.sub(r'(^[A-Za-z]\S*.*.)day\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
        if final_date3 != j :
            M.append(final_date3)
        if final_date4 != j :
            M.append(final_date4)
    G.append(M)
    

aldf["postDate-ontonotes"]=G

In [33]:
G

[['16-Sept-1865'],
 [],
 ['3-October-1865'],
 [],
 ['31-May-1867'],
 ['1-Dec-1866'],
 ['30-May-1864'],
 ['16-Sept-1865'],
 ['16-Nov-1866'],
 ['16-Sept-1865'],
 ['4-November-1867'],
 ['4-November-1867'],
 ['2-Oct-1865'],
 [],
 ['31-December-1867'],
 ['19-June-1865', '19-June-1865'],
 ['4-Nov-1865'],
 ['The 9-November-1866',
  '9-The-1866',
  'The-November-1866',
  'The 9th -November-1866'],
 ['1-December-1866', '1-Dec-1866'],
 ['25-June-1864'],
 ['20-October-1866'],
 ['Third-December-1866', 'Third -December-1866'],
 ['Twenty-July-1867', 'Twenty fourth -July-1867'],
 ['19-May-1865', 'Nineteenth-May-1865', 'Nineteenth -May-1865'],
 [],
 ['twentieth -October-1866'],
 ['15-Janry-1868'],
 ['1-July-1867'],
 [],
 [],
 [],
 ['3-Aug-67'],
 ['3-August-1867'],
 ['15-Dec-67'],
 [],
 [],
 [],
 ['15-Apr-67'],
 [],
 ['23-July-67'],
 ['23-July-1867'],
 ['26-July-1867'],
 ['25-July-1867'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['29-Aug-66'],
 [],
 ['1-Nov-1866'],
 [],
 ['9-Nov-66'],
 [],
 [],
 ['26-July-1867']

**great !**

# best result obtained 

***Person***

****

***Organization***

****

***GPE and location***

****

**Date**

****

In [34]:
dflarge.head()

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,I I C Meekins of Currituck County State of Nor...,"[I I C Meekins, God, Isaac C Meekins, E H Walk...","[Currituck County State of North Carolina, Uni...",[]
1,Department of Virginia North Carolina 1864 tak...,[],"[North Carolina, United States, States]",[Department of Virginia]
2,I W H Pearce of Craven County State of NorthCa...,"[W H Pearce, Almighty, God, Jm E Ampetts, A B ...","[Craven County State of NorthCarolina, United ...",[]
3,State of North Carolina County I solemnly swea...,[God],"[State of North Carolina County, United States]",[]
4,State of North Carolina Wake County I SE Allen...,"[SE Allen, God SE Allen Sworn, McAlpine]","[State of North Carolina, Wake County, United ...",[Res Corp]


In [35]:
dfner.head()

,text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish
0,I I C Meekins of Currituck County State of Nor...,[John B Etheridge],[],[]
1,Department of Virginia North Carolina 1864 tak...,[],[United States],[]
2,I W H Pearce of Craven County State of NorthCa...,"[H Pearce, B Watson]",[],[]
3,State of North Carolina County I solemnly swea...,[],[United States],[]
4,State of North Carolina Wake County I SE Allen...,[],[United States],[]


In [36]:
aldf["Person-NerEngLarge"]=dflarge["PERSON-nerenglarge"]
aldf["LOC-NerEngLarge"]=dflarge["LOC-nerenglarge"]
aldf["ORG-NerEngLarge"]=dflarge["ORG-nerenglarge"]
aldf["Person-NerEng"]=dfner["PERSON-nerenglish"]
aldf["LOC-NerEng"]=dfner["LOC-nerenglish"]
aldf["ORG-NerEng"]=dfner["ORG-nerenglish"]

In [37]:
aldf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Person-NerEngLarge,LOC-NerEngLarge,ORG-NerEngLarge,Person-NerEng,LOC-NerEng,ORG-NerEng
0,I I C Meekins of Currituck County State of Nor...,"[I I C Meekins, Isaac C Meekins, E H Walker JP...",[North Carolina],[],[16 day of Sept AD 1865],[16-Sept-1865],"[I I C Meekins, God, Isaac C Meekins, E H Walk...","[Currituck County State of North Carolina, Uni...",[],[John B Etheridge],[],[]
1,Department of Virginia North Carolina 1864 tak...,[],"[North Carolina, United States]",[Department of Virginia],[1864],[],[],"[North Carolina, United States, States]",[Department of Virginia],[],[United States],[]
2,I W H Pearce of Craven County State of NorthCa...,"[W H Pearce, Watson]",[],[],[3rd day of October AD 1865],[3-October-1865],"[W H Pearce, Almighty, God, Jm E Ampetts, A B ...","[Craven County State of NorthCarolina, United ...",[],"[H Pearce, B Watson]",[],[]
3,State of North Carolina County I solemnly swea...,[],"[North Carolina County, United States]",[],[],[],[God],"[State of North Carolina County, United States]",[],[],[United States],[]
4,State of North Carolina Wake County I SE Allen...,[Allen],"[North Carolina, Wake County, United States]",[],[31st day of May 1867],[31-May-1867],"[SE Allen, God SE Allen Sworn, McAlpine]","[State of North Carolina, Wake County, United ...",[Res Corp],[],[United States],[]


In [38]:
aldf.to_csv("result.csv", index=False)

In [39]:
allPerson=[]
for i in aldf["PERSON-ontonotes"]:
            allPerson.append(i)
allDatess=[]
for i in aldf["Date-ontonotes"]:
            allDatess.append(i)
allpostdates=[]
for i in aldf["postDate-ontonotes"]:
            allpostdates.append(i)
allGPE=[]
for i in aldf["GPE-ontonotes"]:
            allGPE.append(i)
alORGl=[]
for i in aldf["ORG-ontonotes"]:
            alORGl.append(i)

****

In [40]:
Personlarge=[]
for i in dflarge["PERSON-nerenglarge"]:
            Personlarge.append(i)

ORGlarge=[]
for i in dflarge["ORG-nerenglarge"]:
            ORGlarge.append(i)
LOClarge=[]
for i in dflarge["LOC-nerenglarge"]:
            LOClarge.append(i)